## List of observations from the Butler Registry

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-04-29
- last update : 2025-04-29
- last update : 2025-05-07 : add LATISS

In [ ]:
import lsst.pipe.base

print(lsst.pipe.base.__version__)

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

### Choose instrument

In [ ]:
# instrument = "LSSTCam"
instrument = "LSSTComCam"
# instrument = "LATISS"

### Choose option

In [ ]:
FLAG_SAVE_LOGBOOK = False

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- - Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LSSTComCamSim":
    repo = "/repo/main"
    collection_validation = "LSSTComCamSim/*"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "ops_rehersal_prep_2k_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LATISS":
    repo = "/repo/main"
    # collection_validation = instrument + "/runs/quickLook"
    collection_validation = instrument + "/raw/all"
    date_start = 20221001
    date_selection = 20221001
    skymapName = "latiss_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object

In [ ]:
# skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
try:
    skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)
except Exception as inst:
    print(type(inst))  # the exception type
    print(inst.args)  # arguments stored in .args
    print(inst)  # __str__ allows args to be printed directly,

## Dump registry into a pandas dataframe

- Faster method to decode the registry in pandas dataframe : first save deconded filed into a list of fields and then flush the whole list in pandas instead of row by row
- Be carefull the registry variable change in name and type perhaps depending on DM_version

In [ ]:
print(where_clause_date)

In [ ]:
columns = [
    "id",
    "obs_id",
    "day_obs",
    "seq_num",
    "time_start",
    "time_end",
    "type",
    "target",
    "filter",
    "zenith_angle",
    "expos",
    "ra",
    "dec",
    "skyangle",
    "azimuth",
    "zenith",
    "science_program",
    "jd",
    "mjd",
]

In [ ]:
df_exposure = pd.DataFrame(
    {
        "id": pd.Series(dtype="int"),
        "obs_id": pd.Series(dtype="int"),
        "day_obs": pd.Series(dtype="int"),
        "seq_num": pd.Series(dtype="int"),
        "time_start": pd.Series(dtype="str"),  # ou 'datetime64[ns]' si c’est un datetime
        "time_end": pd.Series(dtype="str"),  # idem
        "type": pd.Series(dtype="str"),
        "target": pd.Series(dtype="str"),
        "filter": pd.Series(dtype="str"),
        "zenith_angle": pd.Series(dtype="float"),
        "expos": pd.Series(dtype="float"),  # ou 'int' selon le cas
        "ra": pd.Series(dtype="float"),
        "dec": pd.Series(dtype="float"),
        "skyangle": pd.Series(dtype="float"),
        "azimuth": pd.Series(dtype="float"),
        "zenith": pd.Series(dtype="float"),
        "science_program": pd.Series(dtype="str"),
        "jd": pd.Series(dtype="float"),
        "mjd": pd.Series(dtype="float"),
    }
)

In [ ]:
# save the data array in rows before saving in pandas dataframe
rows = []
for count, info in enumerate(registry.queryDimensionRecords("exposure", where=where_clause_date)):
    try:
        jd_start = info.timespan.begin.value
        jd_end = info.timespan.end.value
        the_Time_start = Time(jd_start, format="jd", scale="utc")
        the_Time_end = Time(jd_end, format="jd", scale="utc")
        mjd_start = the_Time_start.mjd
        mjd_end = the_Time_end.mjd
        isot_start = the_Time_start.isot
        isot_end = the_Time_end.isot

        if count == 0:
            print("===== Time Conversion Debug Info =====")
            print(f"JD start      : {jd_start} (type: {type(jd_start)})")
            print(f"JD end        : {jd_end} (type: {type(jd_end)})")
            print(f"MJD start     : {mjd_start} (type: {type(mjd_start)})")
            print(f"MJD end       : {mjd_end} (type: {type(mjd_end)})")
            print(f"ISOT start    : {isot_start} (type: {type(isot_start)})")
            print(f"ISOT end      : {isot_end} (type: {type(isot_end)})")
            print("=======================================")

        # put row in a dictionnary before stacking
        row = {
            "id": info.id,
            "obs_id": info.obs_id,
            "day_obs": info.day_obs,
            "seq_num": info.seq_num,
            "time_start": isot_start,
            "time_end": isot_end,
            "type": info.observation_type,
            "target": info.target_name,
            "filter": info.physical_filter,
            "zenith_angle": info.zenith_angle,
            "expos": info.exposure_time,  # Exemple : adapter selon ton objet
            "ra": info.tracking_ra,
            "dec": info.tracking_dec,
            "skyangle": info.sky_angle,
            "azimuth": info.azimuth,
            "zenith": info.zenith_angle,
            "science_program": info.science_program,
            "jd": float(jd_start),
            "mjd": float(mjd_start),
        }
        rows.append(row)

    except ValueError as e:
        print(f"Erreur de valeur : {e}")
    except FileNotFoundError as e:
        print(f"Fichier introuvable : {e}")
    except Exception as e:
        print(f"Erreur inattendue : {type(e).__name__} - {e}")
        print(f">>>   Unexpected error at row {count}:", sys.exc_info()[0])
        traceback.print_exc()  # affiche la stack trace complète

In [ ]:
# Création finale du DataFrame
df_exposure = pd.DataFrame(rows)

In [ ]:
# df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

## Select science exposures

In [ ]:
df_science = df_exposure[df_exposure.type == "science"]
df_science.reset_index(drop=True, inplace=True)

In [ ]:
df_science = df_science.copy()
df_science["band"] = df_science["filter"].apply(lambda x : x.split("_")[0])

In [ ]:
df_science

## Save in csv file

In [ ]:
id_min = df_science_id_min = df_science.id.min()
id_max = df_science_id_max = df_science.id.max()
nvisits = len(df_science)

In [ ]:
print(f"\t >> Science : {nvisits} visits ==> first visit = {id_min}, last visit = {id_max}")

In [ ]:
#! pip install --user openpyxl

In [ ]:
if FLAG_SAVE_LOGBOOK:
    output_file_csv = f"instrument_logbook_{id_min}-{id_max}.csv"
    output_file_xlsx = f"instrument_loggbook_{id_min}-{id_max}.xlsx"
    df_science.to_csv(output_file_csv)